In [ ]:
"""
Filename: Data_Preprocessing.ipynb
Description: Files that Preprocess Electric and Water data in N Days (N / 7 Weeks) Units
Author: Minha Lee
Date: 2024-11-13

Last Updata :
- Slicing the Water and Electric Data by 14 Days (2 Weeks) (24.11.13)
- Slicing the Water and ELectric Data by 28 Days (4 Weeks) (24.11.14)
"""

In [ ]:
# Import the Modules

import pandas as pd

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torchmetrics.regression import R2Score

In [ ]:
# Set a Data Path
DATA_PATH = '../Data/수도전기사용량_3개월분_수정본.xlsx'

# Read Water and Electric Data (Time Series)
df1 = pd.read_excel(DATA_PATH, header = 1)
df2 = pd.read_excel(DATA_PATH, header = 1, sheet_name = '수도사용량_3개월분')

df1

Unnamed: 0  2023-06-01 00:00:00  2023-06-01 00:00:00.1  \
0            세대                 1.00                   2.00   
1            1호                 1.23                   1.45   
2            2호                 1.35                   1.35   
3            3호                 1.36                   1.44   
4            4호                 1.23                   1.44   
...         ...                  ...                    ...   
997        997호                 1.29                   1.69   
998        998호                 1.44                   1.64   
999        999호                 1.23                   1.45   
1000      1000호                 1.45                   1.64   
1001        NaN                  NaN                    NaN   

      2023-06-01 00:00:00.2  2023-06-01 00:00:00.3  2023-06-02 00:00:00  \
0                      3.00                   4.00                 1.00   
1                      1.73                   1.62                 1.31   
2                      1.35                   1.34                 1.36   
3                      1.42                   1.36                 1.36   
4                      1.73                   1.61                 1.33   
...                     ...                    ...                  ...   
997                    1.57                   1.26                 1.36   
998                    1.55                   1.45                 1.56   
999                    1.73                   1.62                 1.31   
1000                   1.79                   1.34                 1.35   
1001                    NaN                    NaN                  NaN   

      2023-06-02 00:00:00.1  2023-06-02 00:00:00.2  2023-06-02 00:00:00.3  \
0                      2.00                   3.00                   4.00   
1                      1.48                   1.74                   1.71   
2                      1.42                   1.41                   1.38   
3                      1.42                   1.43                   1.39   
4                      1.47                   1.73                   1.71   
...                     ...                    ...                    ...   
997                    1.64                   1.59                   1.48   
998                    1.45                   1.71                   1.63   
999                    1.48                   1.74                   1.71   
1000                   1.46                   1.49                   1.33   
1001                    NaN                    NaN                    NaN   

      2023-06-03 00:00:00  ...  Unnamed: 385  Unnamed: 386  Unnamed: 387  \
0                    1.00  ...           NaN           NaN           NaN   
1                    1.30  ...           NaN           NaN           NaN   
2                    1.36  ...           NaN           NaN           NaN   
3                    1.32  ...           NaN           NaN           NaN   
4                    1.32  ...           NaN           NaN           NaN   
...                   ...  ...           ...           ...           ...   
997                  1.72  ...           NaN           NaN           NaN   
998                  1.34  ...           NaN           NaN           NaN   
999                  1.32  ...           NaN           NaN           NaN   
1000                 1.44  ...           NaN           NaN           NaN   
1001                  NaN  ...           NaN           NaN           NaN   

      Unnamed: 388  Unnamed: 389  Unnamed: 390  Unnamed: 391  Unnamed: 392  \
0              NaN           NaN           NaN           NaN           NaN   
1              NaN           NaN           NaN           NaN           NaN   
2              NaN           NaN           NaN           NaN           NaN   
3              NaN           NaN           NaN           NaN           NaN   
4              NaN           NaN           NaN           NaN           NaN   
...            ...           ...         

In [3]:
df2

Unnamed: 0  2023-06-01 00:00:00  2023-06-01 00:00:00.1  \
0            세대                    1                      2   
1            1호                   30                    120   
2            2호                   58                    218   
3            3호                   28                    184   
4            4호                   14                    169   
...         ...                  ...                    ...   
996        996호                  188                     95   
997        997호                  188                     95   
998        998호                  218                    350   
999        999호                   43                    136   
1000      1000호                   10                     68   

      2023-06-01 00:00:00.2  2023-06-01 00:00:00.3  2023-06-02 00:00:00  \
0                         3                      4                    1   
1                       210                    410                   32   
2                       290                    105                   10   
3                       150                     38                   62   
4                        89                     55                   55   
...                     ...                    ...                  ...   
996                      46                    139                  214   
997                      46                    139                  214   
998                      38                    183                  190   
999                     189                     84                   54   
1000                     38                    263                   78   

      2023-06-02 00:00:00.1  2023-06-02 00:00:00.2  2023-06-02 00:00:00.3  \
0                         2                      3                      4   
1                       184                    180                    260   
2                        68                     38                    263   
3                       151                    140                    145   
4                       130                    135                    138   
...                     ...                    ...                    ...   
996                     247                     53                    160   
997                     247                     53                    160   
998                     230                     98                    143   
999                     141                    188                     95   
1000                    158                    290                    300   

      2023-06-03 00:00:00  ...  Unnamed: 598  Unnamed: 599  Unnamed: 600  \
0                       1  ...           NaN           NaN           NaN   
1                      35  ...           NaN           NaN           NaN   
2                      78  ...           NaN           NaN           NaN   
3                      78  ...           NaN           NaN           NaN   
4                      28  ...           NaN           NaN           NaN   
...                   ...  ...           ...           ...           ...   
996                   210  ...           NaN           NaN           NaN   
997                   210  ...           NaN           NaN           NaN   
998                   253  ...           NaN           NaN           NaN   
999                    46  ...           NaN           NaN           NaN   
1000                  280  ...           NaN           NaN           NaN   

      Unnamed: 601  Unnamed: 602  Unnamed: 603  Unnamed: 604  Unnamed: 605  \
0              NaN           NaN           NaN           NaN           NaN   
1              NaN           NaN           NaN           NaN           NaN   
2              NaN           NaN           NaN           NaN           NaN   
3              NaN           NaN           NaN           NaN           NaN   
4              NaN           NaN           NaN           NaN           NaN   
...            ...           ...         

In [ ]:
# Drop Not Available Contents
df1.drop(1001, axis = 0, inplace = True)

df1.dropna(axis = 1, inplace = True)
df2.dropna(axis = 1, inplace = True)

df1

Unnamed: 0  2023-06-01 00:00:00  2023-06-01 00:00:00.1  \
0            세대                 1.00                   2.00   
1            1호                 1.23                   1.45   
2            2호                 1.35                   1.35   
3            3호                 1.36                   1.44   
4            4호                 1.23                   1.44   
...         ...                  ...                    ...   
996        996호                 1.45                   1.73   
997        997호                 1.29                   1.69   
998        998호                 1.44                   1.64   
999        999호                 1.23                   1.45   
1000      1000호                 1.45                   1.64   

      2023-06-01 00:00:00.2  2023-06-01 00:00:00.3  2023-06-02 00:00:00  \
0                      3.00                   4.00                 1.00   
1                      1.73                   1.62                 1.31   
2                      1.35                   1.34                 1.36   
3                      1.42                   1.36                 1.36   
4                      1.73                   1.61                 1.33   
...                     ...                    ...                  ...   
996                    1.62                   1.31                 1.48   
997                    1.57                   1.26                 1.36   
998                    1.55                   1.45                 1.56   
999                    1.73                   1.62                 1.31   
1000                   1.79                   1.34                 1.35   

      2023-06-02 00:00:00.1  2023-06-02 00:00:00.2  2023-06-02 00:00:00.3  \
0                      2.00                   3.00                   4.00   
1                      1.48                   1.74                   1.71   
2                      1.42                   1.41                   1.38   
3                      1.42                   1.43                   1.39   
4                      1.47                   1.73                   1.71   
...                     ...                    ...                    ...   
996                    1.74                   1.71                   1.30   
997                    1.64                   1.59                   1.48   
998                    1.45                   1.71                   1.63   
999                    1.48                   1.74                   1.71   
1000                   1.46                   1.49                   1.33   

      2023-06-03 00:00:00  ...  2023-08-29 00:00:00.2  2023-08-29 00:00:00.3  \
0                    1.00  ...                   3.00                   4.00   
1                    1.30  ...                   1.66                   1.64   
2                    1.36  ...                   1.59                   1.48   
3                    1.32  ...                   1.64                   1.79   
4                    1.32  ...                   1.73                   1.71   
...                   ...  ...                    ...                    ...   
996                  1.49  ...                   1.68                   1.36   
997                  1.72  ...                   1.46                   1.36   
998                  1.34  ...                   1.79                   1.35   
999                  1.32  ...                   1.49                   1.25   
1000                 1.44  ...                   1.66                   1.35   

      2023-08-30 00:00:00  2023-08-30 00:00:00.1  2023-08-30 00:00:00.2  \
0                    1.00                   2.00                   3.00   
1                    1.27                   1.29                   1.69   
2                    1.41                   1.47                   1.54   
3                    1.34                   1.35                   1.46   
4                    1.32                   1.49                   1.94   
...        

In [6]:
df2.info

<bound method DataFrame.info of      Unnamed: 0  2023-06-01 00:00:00  2023-06-01 00:00:00.1  \
0            세대                    1                      2   
1            1호                   30                    120   
2            2호                   58                    218   
3            3호                   28                    184   
4            4호                   14                    169   
...         ...                  ...                    ...   
996        996호                  188                     95   
997        997호                  188                     95   
998        998호                  218                    350   
999        999호                   43                    136   
1000      1000호                   10                     68   

      2023-06-01 00:00:00.2  2023-06-01 00:00:00.3  2023-06-02 00:00:00  \
0                         3                      4                    1   
1                       210                    410           

In [ ]:
# Drop Unnecessary Indexes
df1.drop(df1.columns[0], axis = 1, inplace = True)
df2.drop(df2.columns[0], axis = 1, inplace = True)

In [ ]:
# Change the columns' names by six hours
column_name = pd.date_range('2023-06-01', '2023-09-01', freq = '6H')
column_name = column_name[:-1]

column_name

DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 06:00:00',
               '2023-06-01 12:00:00', '2023-06-01 18:00:00',
               '2023-06-02 00:00:00', '2023-06-02 06:00:00',
               '2023-06-02 12:00:00', '2023-06-02 18:00:00',
               '2023-06-03 00:00:00', '2023-06-03 06:00:00',
               ...
               '2023-08-29 12:00:00', '2023-08-29 18:00:00',
               '2023-08-30 00:00:00', '2023-08-30 06:00:00',
               '2023-08-30 12:00:00', '2023-08-30 18:00:00',
               '2023-08-31 00:00:00', '2023-08-31 06:00:00',
               '2023-08-31 12:00:00', '2023-08-31 18:00:00'],
              dtype='datetime64[ns]', length=368, freq='6H')

In [9]:
df1.columns = column_name
df2.columns = column_name

df1.columns

DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 06:00:00',
               '2023-06-01 12:00:00', '2023-06-01 18:00:00',
               '2023-06-02 00:00:00', '2023-06-02 06:00:00',
               '2023-06-02 12:00:00', '2023-06-02 18:00:00',
               '2023-06-03 00:00:00', '2023-06-03 06:00:00',
               ...
               '2023-08-29 12:00:00', '2023-08-29 18:00:00',
               '2023-08-30 00:00:00', '2023-08-30 06:00:00',
               '2023-08-30 12:00:00', '2023-08-30 18:00:00',
               '2023-08-31 00:00:00', '2023-08-31 06:00:00',
               '2023-08-31 12:00:00', '2023-08-31 18:00:00'],
              dtype='datetime64[ns]', length=368, freq='6H')

In [10]:
df1[df1.columns[0:12]]

2023-06-01 00:00:00  2023-06-01 06:00:00  2023-06-01 12:00:00  \
0                    1.00                 2.00                 3.00   
1                    1.23                 1.45                 1.73   
2                    1.35                 1.35                 1.35   
3                    1.36                 1.44                 1.42   
4                    1.23                 1.44                 1.73   
...                   ...                  ...                  ...   
996                  1.45                 1.73                 1.62   
997                  1.29                 1.69                 1.57   
998                  1.44                 1.64                 1.55   
999                  1.23                 1.45                 1.73   
1000                 1.45                 1.64                 1.79   

      2023-06-01 18:00:00  2023-06-02 00:00:00  2023-06-02 06:00:00  \
0                    4.00                 1.00                 2.00   
1                    1.62                 1.31                 1.48   
2                    1.34                 1.36                 1.42   
3                    1.36                 1.36                 1.42   
4                    1.61                 1.33                 1.47   
...                   ...                  ...                  ...   
996                  1.31                 1.48                 1.74   
997                  1.26                 1.36                 1.64   
998                  1.45                 1.56                 1.45   
999                  1.62                 1.31                 1.48   
1000                 1.34                 1.35                 1.46   

      2023-06-02 12:00:00  2023-06-02 18:00:00  2023-06-03 00:00:00  \
0                    3.00                 4.00                 1.00   
1                    1.74                 1.71                 1.30   
2                    1.41                 1.38                 1.36   
3                    1.43                 1.39                 1.32   
4                    1.73                 1.71                 1.32   
...                   ...                  ...                  ...   
996                  1.71                 1.30                 1.49   
997                  1.59                 1.48                 1.72   
998                  1.71                 1.63                 1.34   
999                  1.74                 1.71                 1.32   
1000                 1.49                 1.33                 1.44   

      2023-06-03 06:00:00  2023-06-03 12:00:00  2023-06-03 18:00:00  
0                    2.00                 3.00                 4.00  
1                    1.49                 1.94                 1.68  
2                    1.42                 1.48                 1.39  
3                    1.41                 1.49                 1.41  
4                    1.49                 1.94                 1.68  
...                   ...                  ...                  ...  
996                  1.94                 1.68                 1.26  
997                  1.63                 1.37                 1.43  
998                  1.31                 1.48                 1.54  
999                  1.49                 1.94                 1.68  
1000                 1.45                 1.59                 1.36  

[1001 rows x 12 columns]

In [ ]:
# Drop Unnecessary Rows
df1.drop(0, axis = 0, inplace = True)
df2.drop(0, axis = 0, inplace = True)

df1

2023-06-01 00:00:00  2023-06-01 06:00:00  2023-06-01 12:00:00  \
1                    1.23                 1.45                 1.73   
2                    1.35                 1.35                 1.35   
3                    1.36                 1.44                 1.42   
4                    1.23                 1.44                 1.73   
5                    1.22                 1.35                 1.67   
...                   ...                  ...                  ...   
996                  1.45                 1.73                 1.62   
997                  1.29                 1.69                 1.57   
998                  1.44                 1.64                 1.55   
999                  1.23                 1.45                 1.73   
1000                 1.45                 1.64                 1.79   

      2023-06-01 18:00:00  2023-06-02 00:00:00  2023-06-02 06:00:00  \
1                    1.62                 1.31                 1.48   
2                    1.34                 1.36                 1.42   
3                    1.36                 1.36                 1.42   
4                    1.61                 1.33                 1.47   
5                    1.57                 1.31                 1.48   
...                   ...                  ...                  ...   
996                  1.31                 1.48                 1.74   
997                  1.26                 1.36                 1.64   
998                  1.45                 1.56                 1.45   
999                  1.62                 1.31                 1.48   
1000                 1.34                 1.35                 1.46   

      2023-06-02 12:00:00  2023-06-02 18:00:00  2023-06-03 00:00:00  \
1                    1.74                 1.71                 1.30   
2                    1.41                 1.38                 1.36   
3                    1.43                 1.39                 1.32   
4                    1.73                 1.71                 1.32   
5                    1.73                 1.76                 1.33   
...                   ...                  ...                  ...   
996                  1.71                 1.30                 1.49   
997                  1.59                 1.48                 1.72   
998                  1.71                 1.63                 1.34   
999                  1.74                 1.71                 1.32   
1000                 1.49                 1.33                 1.44   

      2023-06-03 06:00:00  ...  2023-08-29 12:00:00  2023-08-29 18:00:00  \
1                    1.49  ...                 1.66                 1.64   
2                    1.42  ...                 1.59                 1.48   
3                    1.41  ...                 1.64                 1.79   
4                    1.49  ...                 1.73                 1.71   
5                    1.44  ...                 1.66                 1.64   
...                   ...  ...                  ...                  ...   
996                  1.94  ...                 1.68                 1.36   
997                  1.63  ...                 1.46                 1.36   
998                  1.31  ...                 1.79                 1.35   
999                  1.49  ...                 1.49                 1.25   
1000                 1.45  ...                 1.66                 1.35   

      2023-08-30 00:00:00  2023-08-30 06:00:00  2023-08-30 12:00:00  \
1                    1.27                 1.29                 1.69   
2                    1.41                 1.47                 1.54   
3                    1.34                 1.35                 1.46   
4                    1.32                 1.49                 1.94   
5                    1.27                 1.29                 1.69   
...                   ...                  ...                  ...   
996                  1.43            

In [ ]:
# Define a Function to Apply Sliding Window Algorithm
def slicing(data, step, max_length, num):

    # Make a Empty DataFrame (Step is a Column's Length)
    df = pd.DataFrame(columns = range(0, step + 1))

    diff = max_length - step

    # Change the Data to 28 Days Units by Using List Slicing
    for i in range(num):
        for j in range(diff):
            df.loc[i * diff + j] = list(data.iloc[i][data.columns[j : j + step + 1]])
    
    return df


In [ ]:
# Apply the Function to Electric Data
electric_df = slicing(df1, 27, 368, 1000)
electric_df

0     1     2     3     4     5     6     7     8     9   ...    18  \
0       1.23  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  ...  1.77   
1       1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  ...  1.68   
2       1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  ...  1.20   
3       1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  ...  1.42   
4       1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  1.39  ...  1.88   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
340995  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.32  1.49  1.94  ...  1.46   
340996  1.73  1.62  1.31  1.48  1.74  1.71  1.32  1.49  1.94  1.68  ...  1.34   
340997  1.62  1.31  1.48  1.74  1.71  1.32  1.49  1.94  1.68  1.36  ...  1.46   
340998  1.31  1.48  1.74  1.71  1.32  1.49  1.94  1.68  1.36  1.43  ...  1.93   
340999  1.48  1.74  1.71  1.32  1.49  1.94  1.68  1.36  1.43  1.66  ...  1.66   

          19    20    21    22    23    24    25    26    27  
0       1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  
1       1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  
2       1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  
3       1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  
4       1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  1.59  
...      ...   ...   ...   ...   ...   ...   ...   ...   ...  
340995  1.34  1.46  1.93  1.66  1.35  1.45  1.69  1.74  1.44  
340996  1.46  1.93  1.66  1.35  1.45  1.69  1.74  1.44  1.64  
340997  1.93  1.66  1.35  1.45  1.69  1.74  1.44  1.64  1.53  
340998  1.66  1.35  1.45  1.69  1.74  1.44  1.64  1.53  1.45  
340999  1.35  1.45  1.69  1.74  1.44  1.64  1.53  1.45  1.56  

[341000 rows x 28 columns]

In [ ]:
# Save a New Electric Data to CSV File
electric_df.to_csv('../Data/electric_df_28_days.csv')

In [ ]:
# Apply the Function to Water Data
water_df = slicing(df2, 27, 368, 1000)
water_df

0    1    2    3    4    5    6    7    8    9   ...   18   19   20  \
0        30  120  210  410   32  184  180  260   35  145  ...  188   95   46   
1       120  210  410   32  184  180  260   35  145  203  ...   95   46  139   
2       210  410   32  184  180  260   35  145  203  216  ...   46  139  204   
3       410   32  184  180  260   35  145  203  216   43  ...  139  204  198   
4        32  184  180  260   35  145  203  216   43  136  ...  204  198   53   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
340995  230  169   47  122  218  350  110  183  190  230  ...   58  218  290   
340996  169   47  122  218  350  110  183  190  230   98  ...  218  290  105   
340997   47  122  218  350  110  183  190  230   98  143  ...  290  105   10   
340998  122  218  350  110  183  190  230   98  143  253  ...  105   10   68   
340999  218  350  110  183  190  230   98  143  253  198  ...   10   68   38   

         21   22   23   24   25   26   27  
0       139  204  198   53  162  210  150  
1       204  198   53  162  210  150   51  
2       198   53  162  210  150   51  169  
3        53  162  210  150   51  169  204  
4       162  210  150   51  169  204  169  
...     ...  ...  ...  ...  ...  ...  ...  
340995  105   10   68   38  263   78  158  
340996   10   68   38  263   78  158  290  
340997   68   38  263   78  158  290  300  
340998   38  263   78  158  290  300  280  
340999  263   78  158  290  300  280  160  

[341000 rows x 28 columns]

In [ ]:
# Save a New Water Data to CSV File
water_df.to_csv('../Data/water_df_28_days.csv')